In [1]:
import os

In [2]:
%pwd

'C:\\Users\\iheba\\IdeaProjects\\Mlops-Regression-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\iheba\\IdeaProjects\\Mlops-Regression-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    expected_hash: str
    status_file:Path

In [6]:
from RegressionProject.constants import *
from RegressionProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            expected_hash=config.expected_hash,
            status_file=config.status_file
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from RegressionProject.logging import logger
import hashlib

In [18]:

from datetime import datetime
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def calculate_file_hash(self, file_path):
        """Calculate the SHA-256 hash of a file."""
        sha256_hash = hashlib.sha256()
        with open(file_path, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()

    def download_file(self):
        print(self.config.source_URL)

        if os.path.exists(self.config.local_data_file):
            logger.info(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")
        else:
            # Download the file
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info:\n{headers}")
        # Validate file hash after download
        if self.validate_file_hash():
            print("File integrity verified.")
            download_date = datetime.now().isoformat()
            self.write_verification_status(True, download_date)
        else:
            print("File integrity verification failed.")
            self.write_verification_status(False, None)
            # Handle error or raise an exception

    def write_verification_status(self, status, download_date):
        """Write verification status and download date to a text file."""
        with open(self.config.status_file, 'w') as f:
            f.write(f"Verification Status: {'True' if status else 'False'}\n")
            if download_date:
                f.write(f"Download Date: {download_date}\n")
        print(f"Verification status written to {self.config.status_file}")
        
    def validate_file_hash(self):
        """Validate the hash of the downloaded file."""
        expected_hash = self.config.expected_hash  # Replace with your actual expected hash
        downloaded_hash = self.calculate_file_hash(self.config.local_data_file)
        return downloaded_hash == expected_hash
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-09 08:02:11,879: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-09 08:02:11,883: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 08:02:11,884: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 08:02:11,885: INFO: common: created directory at: artifacts]
[2024-07-09 08:02:11,886: INFO: common: created directory at: artifacts/data_ingestion]
https://github.com/IhebAI/Datasets/raw/main/student.zip
[2024-07-09 08:02:11,887: INFO: 3206609817: File already exists of size: 7385]
File integrity verified.
Verification status written to artifacts/data_ingestion/status.txt
